In [40]:
import pandas_datareader as web
import pandas as pd
from yahoo_fin import stock_info as si
import datetime as dt
import yfinance as yf
import ta

In [41]:
tickers = si.tickers_nifty50()
# tickers = tickers[250:300]

In [42]:
start = dt.datetime.now() - dt.timedelta(days=365)
end = dt.datetime.now()

In [43]:
sp500_df = yf.download('^NSEI',start,end)
sp500_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-02,17706.199219,17794.599609,17674.800781,17780.000000,17780.000000,271200
2022-02-03,17767.750000,17781.150391,17511.150391,17560.199219,17560.199219,226600
2022-02-04,17590.199219,17617.800781,17462.550781,17516.300781,17516.300781,261400
2022-02-07,17456.300781,17536.750000,17119.400391,17213.599609,17213.599609,265000
2022-02-08,17279.849609,17306.449219,17043.650391,17266.750000,17266.750000,268500
...,...,...,...,...,...,...
2023-01-25,18093.349609,18100.599609,17846.150391,17891.949219,17891.949219,257200
2023-01-27,17877.199219,17884.750000,17493.550781,17604.349609,17604.349609,476300
2023-01-30,17541.949219,17709.150391,17405.550781,17648.949219,17648.949219,432400


In [44]:
sp500_df['Pct Change'] = sp500_df['Adj Close'].pct_change()
sp500_df

,Open,High,Low,Close,Adj Close,Volume,Pct Change
Date,,,,,,,
2022-02-02,17706.199219,17794.599609,17674.800781,17780.000000,17780.000000,271200,NaN
2022-02-03,17767.750000,17781.150391,17511.150391,17560.199219,17560.199219,226600,-0.012362
2022-02-04,17590.199219,17617.800781,17462.550781,17516.300781,17516.300781,261400,-0.002500
2022-02-07,17456.300781,17536.750000,17119.400391,17213.599609,17213.599609,265000,-0.017281
2022-02-08,17279.849609,17306.449219,17043.650391,17266.750000,17266.750000,268500,0.003088
...,...,...,...,...,...,...,...
2023-01-25,18093.349609,18100.599609,17846.150391,17891.949219,17891.949219,257200,-0.012493
2023-01-27,17877.199219,17884.750000,17493.550781,17604.349609,17604.349609,476300,-0.016074
2023-01-30,17541.949219,17709.150391,17405.550781,17648.949219,17648.949219,432400,0.002533


In [45]:
sp500_return = (sp500_df['Pct Change'] + 1).cumprod()[-1]
sp500_return

0.990793069811586

In [46]:
return_list = []
final_df = pd.DataFrame(columns = ['Ticker','Price','Score','PE Ratio','PEG Ratio','SMA 100','SMA 200','52 Weeks Low','52 Weeks High'])   

In [47]:
def file_exists(path):
    """Test whether a path exists.  Returns False for broken symbolic links"""
    import os
    try:
        st = os.stat(path)
    except os.error:
        return False
    return True

In [48]:
for ticker in tickers:
    if(file_exists(f'stock_data/{ticker}.NS.csv') == False):
        try:
            df = yf.download(ticker,start,end)
            df.to_csv(f'stock_data/{ticker}.NS.csv')
            df['Pct Change'] = df['Adj Close'].pct_change()
            stock_return = (df['Pct Change'] + 1).cumprod()[-1]
            return_compared = round((stock_return/sp500_return),2)
            return_list.append(return_compared)
        except Exception as e:
            print(e)
        
    else:
        try:
            df = pd.read_csv(f'stock_data/{ticker}.NS.csv',index_col = 0)
            df['Pct Change'] = df['Adj Close'].pct_change()
            stock_return = (df['Pct Change'] + 1).cumprod()[-1]
            return_compared = round((stock_return/sp500_return),2)
            return_list.append(return_compared)
        except Exception as e:
            print(e)
        

index -1 is out of bounds for axis 0 with size 0


In [49]:
return_list

[1.21,
 0.92,
 1.13,
 0.77,
 0.81,
 1.08,
 1.26,
 1.1,
 1.51,
 0.81,
 1.07,
 0.93,
 1.06,
 1.09,
 1.65,
 0.91,
 1.11,
 1.05,
 1.06,
 1.28,
 0.92,
 0.99,
 1.02,
 2.37,
 0.92,
 0.72,
 0.96,
 0.97,
 0.7]

In [50]:
best_performers = pd.DataFrame(list(zip(tickers,return_list)), columns = ['Ticker','Returns Compared'])
best_performers['Score'] = best_performers['Returns Compared'].rank(pct = True) * 100
best_performers = best_performers[best_performers['Score']>=best_performers['Score'].quantile(0.7)]
best_performers

,Ticker,Returns Compared,Score
0,ADANIENT.NS,1.21,82.758621
2,BAJAJ-AUTO.NS,1.13,79.310345
6,BRITANNIA.NS,1.26,86.206897
7,CIPLA.NS,1.10,72.413793
8,COALINDIA.NS,1.51,93.103448
14,ITC.NS,1.65,96.551724
16,LT.NS,1.11,75.862069
19,NESTLEIND.NS,1.28,89.655172
23,TATACONSUM.NS,2.37,100.000000


In [51]:
final_df = pd.DataFrame(columns = ['Ticker','Price','Score','PE Ratio','PEG Ratio','SMA 100','SMA 200','52 Weeks Low','52 Weeks High'])   

for ticker in best_performers['Ticker']:
    try:
        df = pd.read_csv(f'stock_data/{ticker}.NS.csv',index_col = 0)
        df['SMA 100'] = ta.trend.sma_indicator(df['Adj Close'], window=100, fillna=False)
        df['SMA 200'] = ta.trend.sma_indicator(df['Adj Close'], window=200, fillna=False)
        close = df['Adj Close'][-1]
        pe_ratio = float(si.get_quote_table(ticker)['PE Ratio (TTM)'])
        peg_ratio = float(si.get_stats_valuation(ticker)[1][4])
        sma_100 = df['SMA 100'][-1]
        sma_200 = df['SMA 200'][-1]
        low_52_weeks = round(min(df['Low'][-(52*5):]),2)
        high_52_weeks = round(max(df['High'][-(52*5):]),2)
        score = round(best_performers[best_performers['Ticker']==ticker]['Score'].tolist()[0],2)
        
        condition_1 = close > sma_100 > sma_200
        condition_2 = close > (1.3 * low_52_weeks)
        condition_3 = close > (0.75 * high_52_weeks)
        condition_4 = pe_ratio < 40
        condition_5 = peg_ratio < 2
        
        if(condition_1 and condition_2 and condition_3 and condition_4 and condition_5 ):
            print(f'{ticker} is eligible')
            final_df = final_df.append({
                'Ticker' : ticker,
                'Price' : close,
                'Score' : score,
                'PE Ratio' : pe_ratio,
                'PEG Ratio' : peg_ratio,
                'SMA 100': sma_100,
                'SMA 200' : sma_200,
                '52 Weeks Low' : low_52_weeks,
                '52 Weeks High' : high_52_weeks,
            },ignore_index=True)
        
    except Exception as e:
        print(f'{e} for {ticker}')
        pass

C:\Users\bucky\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
C:\Users\bucky\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)
C:\Users\bucky\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
C:\Users\bucky\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)
C:\Users\bucky\anaconda3\lib\site-pack

In [52]:
final_df.head()

,Ticker,Price,Score,PE Ratio,PEG Ratio,SMA 100,SMA 200,52 Weeks Low,52 Weeks High


In [53]:
final_df = final_df.sort_values(by='Score',ascending = False)
pd.set_option('display.max_columns',20)
final_df

,Ticker,Price,Score,PE Ratio,PEG Ratio,SMA 100,SMA 200,52 Weeks Low,52 Weeks High


In [54]:
final_df.to_csv("final_df_screened_india.csv")